Run this to install required services:

pip install mlflow boto3 torch fastapi uvicorn


In [22]:
import os
import mlflow
import mlflow.pytorch
import torch
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.responses import JSONResponse
import nest_asyncio
import uvicorn
from fastapi import UploadFile, File
from fastapi.responses import JSONResponse
from PIL import Image
import torchvision.transforms as transforms
import io


In [23]:
import nest_asyncio
nest_asyncio.apply()  # Fixes event loop conflicts in Jupyter


In [24]:
app = FastAPI()

class InputData(BaseModel):
    inputs: list

model = None  # global to be loaded in startup

@app.on_event("startup")
def load_latest_model():
    global model
    os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"
    os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"
    os.environ["AWS_ACCESS_KEY_ID"] = "minioadmin"
    os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin"

    from mlflow.tracking import MlflowClient
    client = MlflowClient()
    exp = client.get_experiment_by_name("chexpert-jupyter")
    if exp is None:
        raise ValueError("Experiment 'chexpert' not found.")

    latest_run = sorted(
        client.search_runs(exp.experiment_id),
        key=lambda r: r.info.start_time,
        reverse=True
    )[0]

    print(f"🔄 Loading model from run {latest_run.info.run_id}")
    model_uri = f"runs:/{latest_run.info.run_id}/final_model"
    model = mlflow.pytorch.load_model(model_uri)
    model.eval()

@app.post("/predict")
async def predict_image(file: UploadFile = File(...)):
    try:
        image_bytes = await file.read()
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
        input_tensor = transform(image).unsqueeze(0)

        # 🔹 Fix: Move to model's device
        device = next(model.parameters()).device
        input_tensor = input_tensor.to(device)

        with torch.no_grad():
            outputs = model(input_tensor)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            return {"probabilities": probs.tolist()}

    except Exception as e:
        print("🔥 Prediction error:", e)
        return JSONResponse(status_code=500, content={"error": str(e)})




C:\Users\shaba\AppData\Local\Temp\ipykernel_8552\698748256.py:8: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [ ]:
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [8552]
INFO:     Waiting for application startup.


🔄 Loading model from run 26c8da488ee5473ebd787fce6eda55c2


INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:62466 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:62470 - "POST /predict HTTP/1.1" 200 OK
